PWC-Net-small model training (with cyclical learning rate schedule)
=======================================================

In this notebook we:
- Use a small model (no dense or residual connections), 6 level pyramid, uspample level 2 by 4 as the final flow prediction
- Train the PWC-Net-small model on a mix of the `FlyingChairs` and `FlyingThings3DHalfRes` dataset using a Cyclic<sub>short</sub> schedule of our own
- The Cyclic<sub>short</sub> schedule oscillates between `5e-04` and `1e-05` for 200,000 steps

Below, look for `TODO` references and customize this notebook based on your own needs.

## Reference

[2018a]<a name="2018a"></a> Sun et al. 2018. PWC-Net: CNNs for Optical Flow Using Pyramid, Warping, and Cost Volume. [[arXiv]](https://arxiv.org/abs/1709.02371) [[web]](http://research.nvidia.com/publication/2018-02_PWC-Net%3A-CNNs-for) [[PyTorch (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/PyTorch) [[Caffe (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/Caffe)

In [1]:
"""
pwcnet_train.ipynb

PWC-Net model training.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Tensorboard:
    [win] tensorboard --logdir=E:\\repos\\tf-optflow\\tfoptflow\\pwcnet-sm-6-2-cyclic-chairsthingsmix
    [ubu] tensorboard --logdir=/media/EDrive/repos/tf-optflow/tfoptflow/pwcnet-sm-6-2-cyclic-chairsthingsmix
"""
from __future__ import absolute_import, division, print_function
import sys
from copy import deepcopy

from dataset_base import _DEFAULT_DS_TRAIN_OPTIONS
from dataset_flyingchairs import FlyingChairsDataset
from dataset_flyingthings3d import FlyingThings3DHalfResDataset
from dataset_mixer import MixedDataset
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_TRAIN_OPTIONS

## TODO: Set this first!

In [2]:
# TODO: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    _DATASET_ROOT = 'E:/datasets/'
else:
    _DATASET_ROOT = '/Vol1/dbstore/datasets/'
_FLYINGCHAIRS_ROOT = _DATASET_ROOT + 'FlyingChairs_release'
_FLYINGTHINGS3DHALFRES_ROOT = _DATASET_ROOT + 'FlyingThings3D_release'
    
# TODO: You MUST adjust the settings below based on the number of GPU(s) used for training
# Set controller device and devices
# A one-gpu setup would be something like controller='/device:GPU:0' and gpu_devices=['/device:GPU:0']
# Here, we use a dual-GPU setup, as shown below
gpu_devices = ['/device:GPU:2']
controller = '/device:GPU:0'

# TODO: You MUST adjust this setting below based on the amount of memory on your GPU(s)
# Batch size
batch_size = 18

# Pre-train on `FlyingChairs+FlyingThings3DHalfRes` mix

## Load the dataset

In [3]:
# TODO: You MUST set the batch size based on the capabilities of your GPU(s) 
#  Load train dataset
ds_opts = deepcopy(_DEFAULT_DS_TRAIN_OPTIONS)
ds_opts['in_memory'] = False                          # Too many samples to keep in memory at once, so don't preload them
ds_opts['aug_type'] = 'heavy'                         # Apply all supported augmentations
ds_opts['batch_size'] = batch_size * len(gpu_devices) # Use a multiple of 8; here, 16 for dual-GPU mode (Titan X & 1080 Ti)
ds_opts['crop_preproc'] = (256, 448)                  # Crop to a smaller input size
ds1 = FlyingChairsDataset(mode='train_with_val', ds_root=_FLYINGCHAIRS_ROOT, options=ds_opts)
ds_opts['type'] = 'into_future'
ds2 = FlyingThings3DHalfResDataset(mode='train_with_val', ds_root=_FLYINGTHINGS3DHALFRES_ROOT, options=ds_opts)
ds = MixedDataset(mode='train_with_val', datasets=[ds1, ds2], options=ds_opts)


In [4]:
# Display dataset configuration
ds.print_config()


Dataset Configuration:
  verbose              False
  in_memory            False
  crop_preproc         (256, 448)
  scale_preproc        None
  tb_test_imgs         False
  random_seed          1969
  val_split            0.03
  aug_type             heavy
  aug_labels           True
  fliplr               0.5
  flipud               0.5
  translate            (0.5, 0.05)
  scale                (0.5, 0.05)
  batch_size           18
  type                 into_future
  mode                 train_with_val
  train size           41731
  val size             1292


## Configure the training

In [5]:
# Start from the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_TRAIN_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_dir'] = './pwcnet-sm-6-2-cyclic-chairsthingsmix/'
nn_opts['batch_size'] = ds_opts['batch_size']
nn_opts['x_shape'] = [2, ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 3]
nn_opts['y_shape'] = [ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 2]
nn_opts['use_tf_data'] = True # Use tf.data reader
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# Use the PWC-Net-small model in quarter-resolution mode
nn_opts['use_dense_cx'] = False
nn_opts['use_res_cx'] = False
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 3

In [6]:
# Set the learning rate schedule. This schedule is for a single GPU using a batch size of 8.
# Below,we adjust the schedule to the size of the batch and the number of GPUs.
nn_opts['lr_policy'] = 'cyclic'
nn_opts['cyclic_lr_max'] = 5e-04 # Anything higher will generate NaNs
nn_opts['cyclic_lr_base'] = 1e-05
nn_opts['cyclic_lr_stepsize'] = 20000
nn_opts['max_steps'] = 200000

# Below,we adjust the schedule to the size of the batch and our number of GPUs (2).
nn_opts['max_steps'] = int(nn_opts['max_steps'] * 8 / ds_opts['batch_size'])
nn_opts['cyclic_lr_stepsize'] = int(nn_opts['cyclic_lr_stepsize'] * 8 / ds_opts['batch_size'])

In [7]:
# Instantiate the model and display the model configuration
nn = ModelPWCNet(mode='train_with_val', options=nn_opts, dataset=ds)
nn.print_config()

Building model...
... model built.
Configuring training ops...
... training ops configured.
Initializing model from previous checkpoint ./pwcnet-sm-6-2-cyclic-chairsthingsmix/pwcnet.ckpt-57000 to resume training...

INFO:tensorflow:Restoring parameters from ./pwcnet-sm-6-2-cyclic-chairsthingsmix/pwcnet.ckpt-57000
... model initialized

Model Configuration:
  verbose                True
  ckpt_dir               ./pwcnet-sm-6-2-cyclic-chairsthingsmix/
  max_to_keep            10
  x_dtype                <dtype: 'float32'>
  x_shape                [2, 256, 448, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [256, 448, 2]
  train_mode             train
  adapt_info             None
  sparse_gt_flow         False
  display_step           100
  snapshot_step          1000
  val_step               1000
  val_batch_size         -1
  tb_val_imgs            pyramid
  tb_test_imgs           None
  gpu_devices            ['/device:GPU:2']
  controller             /device:G

## Train the model

In [ ]:
# Train the model
nn.train()

Resume training from step 57001...
Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
2019-04-19 13:03:48 Iter 57100 [Train]: loss=87.17, epe=27.31, lr=0.000036, samples/sec=25.1, sec/step=0.718, eta=6:20:17
2019-04-19 13:04:44 Iter 57200 [Train]: loss=95.55, epe=30.29, lr=0.000037, samples/sec=36.1, sec/step=0.499, eta=4:23:18
2019-04-19 13:05:41 Iter 57300 [Train]: loss=83.09, epe=25.99, lr=0.000037, samples/sec=35.3, sec/step=0.510, eta=4:28:34
2019-04-19 13:06:37 Iter 57400 [Train]: loss=91.28, epe=28.53, lr=0.000038, samples/sec=35.9, sec/step=0.502, eta=4:23:26
2019-04-19 13:07:34 Iter 57500 [Train]: loss=89.14, epe=27.93, lr=0.000039, samples/sec=35.8, sec/step=0.503, eta=4:23:19
2019-04-19 13:08:29 Iter 57600 [Train]: loss=80.98, epe=25.36, lr=0.000039, samples/sec=36.1, sec/step=0.498, eta=4:19:53
2019-04-19 13:09:24 Iter 57700 [Train]: loss=86.39, epe=27.13, lr=0.000040, samp

## Training log

Here are the training curves for the run above:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/loss.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/epe.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/lr.png)

Here are the predictions issued by the model for a few validation samples:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val1.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val2.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val3.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val4.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val5.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val6.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val7.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val8.png)